# Session 4 – Paghahambing ng SLM at LLM

Ihambing ang latency at kalidad ng sample na tugon sa pagitan ng isang Small Language Model at isang mas malaking modelo na tumatakbo gamit ang Foundry Local.


## ⚡ Mabilisang Simula

**Memory-Optimized Setup (Na-update):**
1. Awtomatikong pinipili ng mga modelo ang mga CPU variant (gumagana sa anumang hardware)
2. Gumagamit ng `qwen2.5-3b` sa halip na 7B (nakakatipid ng ~4GB RAM)
3. Awtomatikong pag-detect ng port (walang manual na configuration)
4. Kabuuang RAM na kailangan: ~8GB inirerekomenda (mga modelo + OS)

**Terminal Setup (30 segundo):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

Pagkatapos, patakbuhin ang notebook na ito! 🚀


### Paliwanag: Pag-install ng Dependency
Nag-i-install ng mga pangunahing package (`foundry-local-sdk`, `openai`, `numpy`) na kinakailangan para sa timing at mga kahilingan sa chat. Ligtas na ulitin nang idempotently.


# Scenario
Ihambing ang isang kinatawan na Small Language Model (SLM) sa isang mas malaking modelo gamit ang isang prompt upang ipakita ang mga trade-off:
- **Pagkakaiba sa latency** (segundo sa orasan)
- **Paggamit ng token** (kung available) bilang proxy para sa throughput
- **Halimbawang output na may kalidad** para sa mabilisang pagsusuri
- **Kalkulasyon ng bilis** upang masukat ang mga benepisyo sa performance

**Mga Variable ng Kapaligiran:**
- `SLM_ALIAS` - Maliit na language model (default: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - Mas malaking language model (default: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - Prompt para sa pagsusuri
- `COMPARE_RETRIES` - Bilang ng pagsubok para sa resilience (default: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - Override sa service endpoint (auto-detected kung hindi nakatakda)

**Paano Ito Gumagana (Opisyal na Pattern ng SDK):**
1. **FoundryLocalManager** ang nag-i-initialize at namamahala sa Foundry Local service
2. Ang serbisyo ay awtomatikong nagsisimula kung hindi tumatakbo (walang kinakailangang manual na setup)
3. Ang mga modelo ay awtomatikong na-resolve mula sa mga alias patungo sa mga konkretong ID
4. Pinipili ang mga hardware-optimized na variant (CUDA, NPU, o CPU)
5. Ang OpenAI-compatible na client ang gumagawa ng chat completions
6. Kinukuha ang mga metrics: latency, tokens, kalidad ng output
7. Ang mga resulta ay ikinukumpara upang makalkula ang speedup ratio

Ang micro-comparison na ito ay tumutulong magdesisyon kung kailan ang pag-route sa mas malaking modelo ay makatuwiran para sa iyong use case.

**SDK Reference:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: Gumagamit ng opisyal na pattern mula sa ../samples/workshop_utils.py

**Pangunahing Benepisyo:**
- ✅ Awtomatikong pagtuklas at pag-initialize ng serbisyo
- ✅ Awtomatikong pagsisimula ng serbisyo kung hindi tumatakbo
- ✅ Built-in na pag-resolve ng modelo at caching
- ✅ Hardware optimization (CUDA/NPU/CPU)
- ✅ OpenAI SDK compatibility
- ✅ Matibay na error handling na may retries
- ✅ Lokal na inference (hindi kinakailangan ang cloud API)


## 🚨 Mga Paunang Kailangan: Dapat Naka-on ang Foundry Local!

**Bago patakbuhin ang notebook na ito**, tiyakin na naka-set up ang serbisyo ng Foundry Local:

### Mga Mabilisang Utos (Patakbuhin sa Terminal):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### Mga Alternatibong Modelo (kung hindi available ang default):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **Kung laktawan mo ang mga hakbang na ito**, makakakita ka ng `APIConnectionError` kapag pinatakbo ang mga cell ng notebook sa ibaba.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### Paliwanag: Pangunahing Imports
Nagdadala ng mga utility para sa timing at mga kliyente ng Foundry Local / OpenAI na ginagamit upang kunin ang impormasyon ng modelo at magsagawa ng mga chat completion.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### Paliwanag: Mga Alias at Pag-set up ng Prompt
Nagbibigay ng mga alias na maaaring i-configure sa environment para sa maliit at mas malaking modelo, pati na rin ang isang prompt para sa paghahambing. Ayusin ang mga environment variable upang mag-eksperimento sa iba't ibang pamilya ng modelo o mga gawain.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 Memory-Optimized Configuration

**Ang notebook na ito ay gumagamit ng mga modelong memory-efficient bilang default:**
- `phi-4-mini` → ~4GB RAM (Ang Foundry Local ay awtomatikong pumipili ng CPU variant)
- `qwen2.5-3b` → ~3GB RAM (sa halip na 7B na nangangailangan ng ~7GB+)

**Auto-Detection ng Port:**
- Maaaring gumamit ang Foundry Local ng iba't ibang port (karaniwang 55769 o 59959)
- Ang diagnostic cell sa ibaba ay awtomatikong natutukoy ang tamang port
- Hindi na kailangan ng manual na configuration!

**Kung limitado ang iyong RAM (<8GB), gumamit ng mas maliit pang mga modelo:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### Paliwanag: Execution Helpers (Foundry SDK Pattern)
Gumagamit ng opisyal na Foundry Local SDK pattern tulad ng nakadokumento sa mga halimbawa ng Workshop:

**Diskarte:**
- **FoundryLocalManager** - Nag-i-initialize at namamahala sa Foundry Local service
- **Auto-Detection** - Awtomatikong natutuklasan ang endpoint at pinangangasiwaan ang lifecycle ng serbisyo
- **Model Resolution** - Inaayos ang mga alias sa buong model ID (hal., phi-4-mini → phi-4-mini-instruct-cpu)
- **Hardware Optimization** - Pinipili ang pinakamahusay na variant para sa magagamit na hardware (CUDA, NPU, o CPU)
- **OpenAI Client** - Nakakonfigura gamit ang endpoint ng manager para sa OpenAI-compatible API access

**Mga Tampok ng Resilience:**
- Exponential backoff retry logic (maaaring i-configure sa pamamagitan ng environment)
- Awtomatikong pagsisimula ng serbisyo kung hindi tumatakbo
- Pag-verify ng koneksyon pagkatapos ng initialization
- Maayos na paghawak ng error na may detalyadong ulat ng error
- Model caching upang maiwasan ang paulit-ulit na initialization

**Istruktura ng Resulta:**
- Pagsukat ng latency (wall clock time)
- Pagsubaybay sa paggamit ng token (kung magagamit)
- Halimbawang output (pinutol para sa readability)
- Mga detalye ng error para sa mga nabigong request

Ang pattern na ito ay gumagamit ng workshop_utils module na sumusunod sa opisyal na SDK pattern.

**SDK Reference:**
- Main Repo: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### Paliwanag: Pre-Flight Self-Test
Nagpapatakbo ng magaan na connectivity check gamit ang FoundryLocalManager para sa parehong modelo. Sini-sigurado nito na:
- Maaaring ma-access ang serbisyo
- Maaaring ma-initialize ang mga modelo
- Ang mga alias ay tumutugma sa aktwal na mga ID ng modelo
- Matatag ang koneksyon bago magsagawa ng paghahambing

Ginagamit ng setup() function ang opisyal na pattern ng SDK mula sa workshop_utils.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ Mabilisang Ayos: Simulan ang Foundry Local mula sa Notebook (Opsyonal)

Kung ipinapakita ng diagnostic sa itaas na hindi tumatakbo ang serbisyo, maaari mong subukang simulan ito mula rito:

**Tandaan:** Mas epektibo ito sa Windows. Sa ibang mga platform, gumamit ng mga terminal command.


### ⚠️ Pag-aayos ng Mga Error sa Koneksyon

Kung nakikita mo ang `APIConnectionError`, maaaring hindi tumatakbo ang Foundry Local service o hindi naka-load ang mga modelo. Subukan ang mga hakbang na ito:

**1. Suriin ang Status ng Serbisyo:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. I-start ang Serbisyo (kung hindi tumatakbo):**
```bash
foundry service start
```

**3. I-load ang Kinakailangang Mga Modelo:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. Siguraduhing Available ang Mga Modelo:**
```bash
foundry model ls
```

**Karaniwang Mga Isyu:**
- ❌ Hindi tumatakbo ang serbisyo → Patakbuhin ang `foundry service start`
- ❌ Hindi naka-load ang mga modelo → Patakbuhin ang `foundry model run <model-name>`
- ❌ Mga conflict sa port → Suriin kung may ibang serbisyo na gumagamit ng port
- ❌ Firewall na nagba-block → Siguraduhing pinapayagan ang mga lokal na koneksyon

**Mabilisang Solusyon:** Patakbuhin ang diagnostic cell sa ibaba bago ang pre-flight check.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ Pre-Flight Check: Pagkakaroon ng Modelo

Ang cell na ito ay nagbe-verify kung maaring ma-access ang parehong modelo sa naka-configure na endpoint bago isagawa ang paghahambing.


### Paliwanag: Paghahambing ng Pagpapatakbo at Pagkolekta ng Resulta
Inuulit ang proseso sa parehong mga alias gamit ang opisyal na pattern ng Foundry SDK:
1. I-initialize ang bawat modelo gamit ang setup() (gumagamit ng FoundryLocalManager)
2. Patakbuhin ang inference gamit ang OpenAI-compatible API
3. Irekord ang latency, mga token, at sample na output
4. Gumawa ng JSON summary na may comparative analysis

Sinusunod nito ang parehong pattern tulad ng mga sample sa Workshop sa session04/model_compare.py.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### Pag-unawa sa Resulta

**Mga Pangunahing Sukatan:**
- **Latency**: Mas mababa, mas maganda - nagpapahiwatig ng mas mabilis na oras ng tugon
- **Tokens**: Mas mataas na throughput = mas maraming tokens ang naproseso
- **Route**: Kumpirmasyon kung aling API endpoint ang ginamit

**Kailan Gagamitin ang SLM kumpara sa LLM:**
- **SLM (Small Language Model)**: Mabilis na tugon, mas mababang paggamit ng resources, maganda para sa mga simpleng gawain
- **LLM (Large Language Model)**: Mas mataas na kalidad, mas mahusay na pangangatwiran, gamitin kapag mahalaga ang kalidad

**Mga Susunod na Hakbang:**
1. Subukan ang iba't ibang prompts upang makita kung paano naaapektuhan ng pagiging kumplikado ang paghahambing
2. Mag-eksperimento sa iba pang pares ng modelo
3. Gamitin ang mga sample ng Workshop router (Session 06) upang matalinong mag-route batay sa pagiging kumplikado ng gawain


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
